In [12]:
NEYNAR_API_KEY="21B5F790-2634-492A-9B91-B56C01C47673"

In [34]:
url = "https://dashboard.alchemy.com/api/webhook-addresses?webhook_id=wh_7095bcxu6cd082vt"

headers = {
    "accept": "application/json",
    "X-Alchemy-Token": "d4m6KmZP-NOXfq4GuwRvBBKsQQmAtk9i"
}

response = requests.get(url, headers=headers)

print(response.text)

{"data":["0x2fa856d3a79fabff523344cbe987c9e2ba5f8879","0x77fbfe78d461980ba158258f8bc0dced937727a8","0x849151d7d0bf1f34b70d5cad5149d28cc2308bf1","0xcce9a28b570946123f392cf1dbfa6d2d5e636a1f","0xd8da6bf26964af9d7eed9e03e53415d37aa96045"],"pagination":{"cursors":{},"total_count":5}}


In [10]:
len(response.json()['next'])

1

In [23]:
def get_follower(cursor=""):
    """
    Fetches followers of @idriss farcaster account from Airstack.

    Args:
        cursor (str): Next page cursor.

    Returns:
        dict: Account name to wallet address mapping.
    """
    
    url = "https://api.neynar.com/v2/farcaster/followers?fid=189333&limit=100"

    base_url = "https://api.neynar.com/v2/farcaster/followers?fid=189333&limit=100"
    url = f"{base_url}&cursor={cursor}" if cursor else base_url
    
    if not NEYNAR_API_KEY:
        raise ValueError("NEYNAR_API_KEY not set")
        

    headers = {"accept": "application/json",
    "x-neynar-experimental": "false",
    "x-api-key": NEYNAR_API_KEY}

    response = requests.get(url, headers=headers, timeout=10)

    response.raise_for_status()
    data = response.json()
    return data



In [24]:
def get_fc_twitter_account_verifications(cursor=""):
    """
    Fetches Twitter verifications from Warpcast API.

    Args:
        cursor (str): Next page cursor.

    Returns:
        dict: fid to twitter name mapping.
    """
    base_url = "https://api.warpcast.com/fc/account-verifications?limit=1000"
    url = f"{base_url}&cursor={cursor}" if cursor else base_url
    response = requests.get(url, timeout=10)
    response.raise_for_status()
    return response.json()

In [30]:
def update_follower():
    """
    Fetches all followers of the @idriss Farcaster account.
    Paginates through all results to ensure all followers are retrieved and returns the latest address mapping.

    Returns:
        dict: A mapping of account names to the latest wallet address (newest by timestamp).
    """
    cursor = ""
    all_followers = {}
    follower_mapping = {}

    while True:
        data = get_follower(cursor)
        
        followers = data["users"]
        page_info = data["next"]
        cursor = page_info["cursor"]

        for follower in followers:
            social = follower['user']
            profile_name = social["username"]
            profile_fid = social["fid"]
            connected_addresses = social["verifications"]
            if profile_name and profile_fid:
                follower_mapping[profile_name] = profile_fid
            if profile_name and connected_addresses:
                evm_addresses = [
                    addr
                    for addr in connected_addresses
                    if addr.startswith("0x")
                ]

                if evm_addresses:
                    latest_evm = evm_addresses[-1]
                    all_followers[profile_name] = {
                        "address": latest_evm,
                        "fid": profile_fid,
                        "twitter": "",
                    }

        if not cursor:
            break
    try:
        verifications = update_twitter_verifications()
        for _, follower_info in all_followers.items():
            fid = follower_info.get("fid", "")

            if fid in verifications:
                follower_info["twitter"] = verifications[fid]
    except Exception:
        print("verifications not updated")
    store_follower(all_followers, follower_mapping)
    return all_followers

In [31]:
update_follower()

verifications not updated
{'avv': {'address': '0x5868d46aec5e5c6871a973bebdc476dc6edac2f7', 'fid': 555713, 'twitter': ''}, 'mbw': {'address': '0x2fdcef473a01db9a33eab432d1d320e7a600afc1', 'fid': 553805, 'twitter': ''}, 'kws': {'address': '0x49e86acaf268d89d97f7cb2a3e94c9a53ca6be3e', 'fid': 537199, 'twitter': ''}, 'hzw': {'address': '0xae287e0da004421980c73fe0048f40e273c5ea2f', 'fid': 537194, 'twitter': ''}, 'ttp': {'address': '0x1f952aed998fb6db92327b715303eebd577c1c6c', 'fid': 537189, 'twitter': ''}, 'uuv': {'address': '0x6f0323b893c23a21c130a4df0fd5e612f240c7a6', 'fid': 537183, 'twitter': ''}, 'hhv': {'address': '0x3a23b739e4be2020f47005243dbb536613ab5c4e', 'fid': 537173, 'twitter': ''}, 'hpa': {'address': '0xad311f460a81de846dd23c17ac5ca2c356b48c32', 'fid': 537167, 'twitter': ''}, 'uhs': {'address': '0xf3662848f5de92894d52588a240beddb9d3edf6b', 'fid': 537157, 'twitter': ''}, 'umw': {'address': '0x0199e231baec4ca2b056e629611cb851bd8e9273', 'fid': 537153, 'twitter': ''}, 'duw': {'addr

{'avv': {'address': '0x5868d46aec5e5c6871a973bebdc476dc6edac2f7',
  'fid': 555713,
  'twitter': ''},
 'mbw': {'address': '0x2fdcef473a01db9a33eab432d1d320e7a600afc1',
  'fid': 553805,
  'twitter': ''},
 'kws': {'address': '0x49e86acaf268d89d97f7cb2a3e94c9a53ca6be3e',
  'fid': 537199,
  'twitter': ''},
 'hzw': {'address': '0xae287e0da004421980c73fe0048f40e273c5ea2f',
  'fid': 537194,
  'twitter': ''},
 'ttp': {'address': '0x1f952aed998fb6db92327b715303eebd577c1c6c',
  'fid': 537189,
  'twitter': ''},
 'uuv': {'address': '0x6f0323b893c23a21c130a4df0fd5e612f240c7a6',
  'fid': 537183,
  'twitter': ''},
 'hhv': {'address': '0x3a23b739e4be2020f47005243dbb536613ab5c4e',
  'fid': 537173,
  'twitter': ''},
 'hpa': {'address': '0xad311f460a81de846dd23c17ac5ca2c356b48c32',
  'fid': 537167,
  'twitter': ''},
 'uhs': {'address': '0xf3662848f5de92894d52588a240beddb9d3edf6b',
  'fid': 537157,
  'twitter': ''},
 'umw': {'address': '0x0199e231baec4ca2b056e629611cb851bd8e9273',
  'fid': 537153,
  'twitt